**About** : This notebook is used to infer models.

In [ ]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

## Initialization

### Imports

In [ ]:
import os
import torch

print(torch.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = "-1"

In [ ]:
import os
import re
import cv2
import sys
import glob
import json
import time
import torch
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.metrics import *

tf.get_logger().setLevel('ERROR')
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

In [ ]:
from utils.logger import Config, upload_to_kaggle

from params import *
from data.preparation import *

from model_zoo.models import define_model
from utils.metrics import *
from utils.torch import load_model_weights, count_parameters
from utils.plots import plot_sample
from inference.main import uniform_soup

## Expes

In [ ]:
MAX_LENS = {
    "torch_12/": 25,
    "torch_15/": 40,
    "torch_16/": 30,
    "torch_18/": 80,
    "torch_19/": 40,
}

In [ ]:
FILES = [  # CV 0.7438 - 0.79
    "../logs/2023-04-28/5/pred_oof_dist_soup.npy",  # 0.7329 / torch_12
    "../logs/2023-04-28/7/pred_oof_dist_soup.npy",  # 0.7359 / torch_19
]

FILES = [  # CV 0.7439  - 0.78
    "../logs/2023-04-25/71/pred_oof_dist.npy",  # 0.7329 / torch_12
    "../logs/2023-04-28/7/pred_oof_dist.npy",  # 0.7359 / torch_19
]

FILES = [  # CV 0.7446 - 0.79+ 572-80_768-25
    "../logs/2023-04-27/17/pred_oof_dist_soup.npy",   # 0.7331  torch_18 576
    "../logs/2023-04-28/9/pred_oof_dist_soup.npy" ,   # 0.7366  torch_12 768
]

FILES = [  # CV 0.7444 - 0.79++ 640-60_768-25n
    "../logs/2023-04-28/9/pred_oof_dist_soup.npy" ,   # 0.7366  torch_12 768
    "../logs/2023-04-29/0/pred_oof_dist_soup.npy",    # 0.7355  torch_19 640
]


FILES = [  # CV 0.7479 - 0.79++  640-60_768-25n (name is wrong)
    "../logs/2023-04-28/7/pred_oof_dist_soup.npy" ,   # 0.7359  torch_19 576
    "../logs/2023-04-30/7/pred_oof_dist_soup.npy",    # 0.7402  torch_12 768 mix norm
]


EXP_FOLDERS = [f.rsplit("/", 1)[0] + "/" for f in FILES]

EXP_FOLDER = EXP_FOLDERS[0]
EXP_FOLDER_2 = EXP_FOLDERS[1]

In [ ]:
config = Config(json.load(open(EXP_FOLDER + "config.json", "r")))
config_2 = Config(json.load(open(EXP_FOLDER_2 + "config.json", "r")))

In [ ]:
df = prepare_data(DATA_PATH, config.processed_folder)

In [ ]:
if "fold" not in df.columns:
    folds = pd.read_csv(config.folds_file)
    df = df.merge(folds, how="left", on=["participant_id", "sequence_id"])

In [ ]:
pred_oof = np.mean([np.load(f) for f in FILES], 0)
df['pred'] = pred_oof.argmax(-1)

score = accuracy(df['target'], pred_oof)
print(f"-> CV acc : {score:.4f}")

## Inference

### Preprocessing

In [ ]:
from tflite.prepro import *

In [ ]:
landmarks = np.concatenate(KEPT_LANDMARKS)
type_embed = np.zeros(1000)
start = 0
for subset, idx in zip(KEPT_LANDMARKS, MAPPING):
    print(subset, idx)
    type_embed[start: start + len(subset)] = idx
    start += len(subset)

type_embed = type_embed[type_embed > 0]

type_embed = np.concatenate([type_embed, np.array([idx] * len(TO_AVG))])

print("\nn_landmarks :", len(type_embed))

In [ ]:
prepro_tf = PreprocessingTF(type_embed, max_len=MAX_LENS[config.processed_folder], model_max_len=config.max_len)
prepro_tf_2 = PreprocessingTF(type_embed, max_len=MAX_LENS[config_2.processed_folder], model_max_len=config_2.max_len)

In [ ]:
print(config.processed_folder, config_2.processed_folder)
print(config.max_len, config_2.max_len)

### Model

In [ ]:
from tflite.models import Model

In [ ]:
N_SOUP = 0
TEACHERS = ['mt' in f for f in FILES]
DISTS = ['dist' in f for f in FILES]
NORMS = [False, True]

print("Teacher :", TEACHERS)
print("Dist :", DISTS)

In [ ]:
models = []

for i, exp_folder in enumerate(EXP_FOLDERS):
    print(f' - {exp_folder} \n')
    config = Config(json.load(open(exp_folder + "config.json", "r")))

    model = Model(
        type_embed,
        embed_dim=config.embed_dim,
        transfo_dim=config.transfo_dim if not DISTS[i] else config.mt_config.get('distill_transfo_dim', 576),
        dense_dim=config.dense_dim if not DISTS[i] else config.mt_config.get('distill_dense_dim', 192),
        transfo_layers=config.transfo_layers,
        transfo_heads=config.transfo_heads,
        drop_rate=config.drop_rate,
        num_classes=config.num_classes,
        max_len=config.max_len,
        normalize=NORMS[i]
    ).cpu().eval()
    
    print(f" -> {count_parameters(model)} params")

    if N_SOUP:
        if TEACHERS[i]:
            weights = [exp_folder + f"{config.name}_teacher_fullfit_0_{ep}.pt" for ep in range(config.epochs - N_SOUP, config.epochs + 1)]
        elif DISTS[i]:
            weights = [exp_folder + f"{config.name}_distilled_fullfit_0_{ep}.pt" for ep in range(config.epochs - N_SOUP, config.epochs + 1)]
        else:
            weights = [exp_folder + f"{config.name}_fullfit_0_{ep}.pt" for ep in range(config.epochs - N_SOUP, config.epochs + 1)]
        print("\n-> Soup :", [w.split('/')[-1] for w in weights])
        model = uniform_soup(model, weights)

    else:
        try:
            if TEACHERS[i]:
                model = load_model_weights(model, exp_folder + f"{config.name}_teacher_fullfit_0.pt")
            elif DISTS[i]:
                model = load_model_weights(model, exp_folder + f"{config.name}_distilled_fullfit_0.pt")
            else:
                model = load_model_weights(model, exp_folder + f"{config.name}_fullfit_0.pt")
        #     model = load_model_weights(model, EXP_FOLDER + f"{config.name}_fullfit_0.pt")
        #     model = load_model_weights(model, EXP_FOLDER + f"{config.name}_teacher_fullfit_0.pt")
#             model = load_model_weights(model, exp_folder + f"{config.name}_0.pt")
        except: # FileNotFoundError:
            try:
                 model = load_model_weights(model, exp_folder + f"{config.name}_distilled_0.pt")
            except:
                print('Not loading weights !')
    
    models.append(model)
    print()

#### Nobuco

In [ ]:
import nobuco
import tensorflow_addons as tfa
from nobuco import ChannelOrder, ChannelOrderingStrategy

In [ ]:
@nobuco.converter(torch.nn.functional.mish, channel_ordering_strategy=ChannelOrderingStrategy.MINIMUM_TRANSPOSITIONS)
def mish(input: torch.Tensor, inplace: bool = False):
    return lambda input, inplace=False: tfa.activations.mish(input)

In [ ]:
@nobuco.converter(torch.Tensor.long, channel_ordering_strategy=ChannelOrderingStrategy.MINIMUM_TRANSPOSITIONS)
def long(input: torch.Tensor, inplace: bool = False):
    return lambda input, inplace=False: tf.cast(input, tf.int64)

In [ ]:
@nobuco.converter(torch.Tensor.mean, channel_ordering_strategy=ChannelOrderingStrategy.FORCE_PYTORCH_ORDER)
def mean(input: torch.Tensor, dim=None, keepdim=False):
    return lambda input, axis: tf.reduce_mean(input, axis=dim)

In [ ]:
@nobuco.converter(torch.Tensor.amax, channel_ordering_strategy=ChannelOrderingStrategy.FORCE_PYTORCH_ORDER)
def amax(input: torch.Tensor, dim=None, keepdim=False):
    return lambda input, axis: tf.reduce_max(input, axis=dim)

In [ ]:
@nobuco.converter(torch.gather, channel_ordering_strategy=ChannelOrderingStrategy.MINIMUM_TRANSPOSITIONS)
def gather(input, dim, index):
    return lambda input, dim, index: tf.gather(input, index)

In [ ]:
path = df['path'][0]
pq, data = load_relevant_data_subset(path)
inp = torch.from_numpy(prepro_tf(data).numpy()).contiguous()
inp.size()

In [ ]:
keras_model = nobuco.pytorch_to_keras(
    models[0],
    args=[inp],
    input_shapes={inp: (None, 5, 100)},
    inputs_channel_order=ChannelOrder.PYTORCH,
    outputs_channel_order=ChannelOrder.TENSORFLOW
)

In [ ]:
keras_model_2 = nobuco.pytorch_to_keras(
    models[1],
    args=[inp],
    input_shapes={inp: (None, 5, 100)},
    inputs_channel_order=ChannelOrder.PYTORCH,
    outputs_channel_order=ChannelOrder.TENSORFLOW
)

In [ ]:
# keras_model.summary()

In [ ]:
df = df[df['fold'] == 0].reset_index(drop=True)
pred_val = df['pred'].values

accuracy(df['target'], pred_val)

In [ ]:
preds = []
for i in tqdm(range(10)):
    path = df['path'][i]
    pq, data = load_relevant_data_subset(path)
    x = prepro_tf(data)
    y = keras_model(x)
    preds.append(y.numpy()[0])
#     break

In [ ]:
accuracy(df['target'].head(len(preds)), preds)

In [ ]:
preds = []
for i in tqdm(range(10)):
    path = df['path'][i]
    pq, data = load_relevant_data_subset(path)
    x = prepro_tf_2(data)
    y = keras_model_2(x)
    preds.append(y.numpy()[0])
#     break

In [ ]:
accuracy(df['target'].head(len(preds)), preds)

### Prepro + model

In [ ]:
class TFLiteModel(tf.keras.Model):
    def __init__(self, prepro, prepro_2, model, model_2):
        super(TFLiteModel, self).__init__()
        self.prepro = prepro
        self.prepro_2 = prepro_2
        self.model = model
        self.model_2 = model_2

    @tf.function(input_signature=[tf.TensorSpec(shape=[None, 543, 3], dtype=tf.float32, name='inputs')])
    def call(self, inputs=None):
        y1 = self.model(self.prepro(tf.cast(inputs, dtype=tf.float32)))
        y2 = self.model_2(self.prepro_2(tf.cast(inputs, dtype=tf.float32)))
        
        y = (tf.nn.softmax(y1, -1) + tf.nn.softmax(y2, -1)) / 2

        return {'outputs': y}

In [ ]:
tflite_keras_model = TFLiteModel(
    prepro_tf,
    prepro_tf_2,
    keras_model,
    keras_model_2,
)

In [ ]:
preds = []
for i in tqdm(range(100)):
# for i in tqdm(range(len(df))):
    path = df['path'][i]
    pq, data = load_relevant_data_subset(path)
    y = tflite_keras_model(data)
    preds.append(y['outputs'].numpy()[0])

In [ ]:
accuracy(df['target'].head(len(preds)), preds)

In [ ]:
accuracy(df['target'].head(len(preds)), pred_val[:len(preds)])

### TfLite

In [ ]:
ENS_NAME = "640-60_768-25n_nosoup"

OUT_FOLDER = "../output/ens/" + ENS_NAME + "/"
os.makedirs(OUT_FOLDER, exist_ok=True)

In [ ]:
tflite_keras_model.save(OUT_FOLDER + 'model_keras')

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(OUT_FOLDER + "model_keras")

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

# converter.target_spec.supported_ops = [
#     tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
#     tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
# ]

tflite_model = converter.convert()

with open(OUT_FOLDER + 'model.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
from tflite_runtime.interpreter import Interpreter

interpreter = Interpreter(OUT_FOLDER + "model.tflite")

prediction_fn = interpreter.get_signature_runner("serving_default")

output = prediction_fn(inputs=data)
output['outputs'].max()

In [ ]:
preds = []
times = []
# for i in tqdm(range(len(df['path']))):
for i in tqdm(range(100)):
    path = df['path'][i]
    name = f"{path.split('/')[-2]}_{path.split('/')[-1].split('.')[0]}.npy"

    pq, data = load_relevant_data_subset(path)

    t0 = time.time()
    output = prediction_fn(inputs=data)
    t1 = time.time()

    preds.append(output['outputs'])
    times.append((t1 - t0) * 1000)
    
#     break

In [ ]:
accuracy(df['target'].head(len(preds)), preds)

In [ ]:
accuracy(df['target'].head(len(preds)), pred_val[:len(preds)])

In [ ]:
print(f'-> Runtime : {np.mean(times) :.1f}ms')

if np.mean(times) > 100:
    print("\n WARNING ! Runtime must be < 100 ms !")

### Size & upload

In [ ]:
size = os.path.getsize(OUT_FOLDER + 'model.tflite') / np.power(1024, 2)
print(f"-> Model size : {size:.3f} Mo")

assert size < 40, "Model size must be < 40 Mo !"

In [ ]:
upload_to_kaggle([OUT_FOLDER], "/workspace/datasets/islr_weights_1/", "ISLR Models", update_folders=False)

Done ! 